<b>EnergyUsagePrediction.feature_eng.asum.v1_0_8.ipynb</b>
<br/>For my use case "Energy usage prediction based on historical weather and energy usage data.". The original dataset  can be downloaded from <a href="https://www.kaggle.com/taranvee/smart-home-dataset-with-weather-information">kaggle</a>
<br/>The dataset used in this step (feature engineering) has already been transformed in the ETL step.
<br/>Data exploration is described/performed in "EnergyUsagePrediction.data_exp.asum.1_0_5.Ipynb"
<br/>ETL is described/performed in "EnergyUsagePrediction.etl.asum.1_0_8.Ipynb"
<br/>
<br/>This task transforms input columns of various relations into additional columns to improve model performance. 
<br/>A subset of those features can be created in an initial task (for example, one-hot encoding of categorical variables or normalization of numerical variables).
<br/>Some others require business understanding or multiple iterations to be considered. 
<br/>This task is one of those benefiting the most from the highly iterative nature of this method.
<br/>
<br/>In this task I will normalize the data columns in order to be easier to use by machine learning algorithms
<br/>I will apply one hot encoding to enum like text columns (columns with just a few text items which indicate a category/type/state)
<br/>Additional features will be added like: day of year, minute of day. 
<br/>Load <i>smart-home-dataset-with-weather-information_filtered.csv</i> file into pandas dataframe


In [ ]:
import types
import numpy as np
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_x = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='[credentials]',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_x.get_object(Bucket='xyz',Key='smart-home-dataset-with-weather-information_post_etl.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()



For usability we define constants for the labels.

In [2]:
df.dtypes

Unnamed: 0                 int64
Timestamp                  int64
TotalUsage_kW            float64
Generated_kW             float64
HouseOverall_kW          float64
DishWasher_kW            float64
Furnace1_kW              float64
Furnace2_kW              float64
HomeOffice_kW            float64
Fridge_kW                float64
WineCellar_kW            float64
GarageDoor_kW            float64
KitchenDevice12_kW       float64
KitchenDevice14_kW       float64
KitchenDevice38_kW       float64
Barn_kW                  float64
Well_kW                  float64
Microwave_kW             float64
Living room_kW           float64
Solar_kW                 float64
Temperature_F            float64
WeatherIndicator          object
Humidity                 float64
Visibility               float64
WeatherSummary            object
ApparentTemperature_F    float64
Pressure_hPa             float64
WindSpeed                float64
cloudCover                object
WindBearing                int64
PrecipInte

In [3]:
lbTimestamp = 'Timestamp'
lbTotalEneryUsage = 'TotalUsage_kW'
lbEneryGeneration = 'Generated_kW'
lbEneryUsageHouseOverall = 'HouseOverall_kW'
lbDishwasherUsage = 'DishWasher_kW'
lbFurnace1Usage = 'Furnace1_kW'
lbFurnace2Usage = 'Furnace2_kW'
lbHomeOfficeUsage = 'HomeOffice_kW'
lbFridgeUsage = 'Fridge_kW'
lbWineUsage = 'WineCellar_kW'
lbGarageDoorUsage = 'GarageDoor_kW'
lbKitchen12Usage = 'KitchenDevice12_kW'
lbKitchen14Usage = 'KitchenDevice14_kW'
lbKitchen38Usage = 'KitchenDevice38_kW'
lbBarnUsage = 'Barn_kW'
lbWellUsage = 'Well_kW'
lbMicrowaveUsage = 'Microwave_kW'
lbLivingRoomUsage = 'Living room_kW'
lbSolarGenerated = 'Solar_kW'
lbTemperature = 'Temperature_F'

#Textial weather indicators: clear-day, clear-night, cloudy, fog, partly-cloudy-day, partly-cloudy-night, rain, snow, wind
lbWeatherIndicator = 'WeatherIndicator'

#Humidity range [0,1]
lbHumidity = 'Humidity'

#Visibility range [0,10]
lbVisibility = 'Visibility'

#WeatherSummary: Breezy, Breezy and mostly cloudy, clear, drizzle, dry , flurries, flurries and breezy, foggy, heavy snow, light rain, light snow, mostly cloudy, overcast, partly cloudy, rain, rain and breezy, snow
lbWeatherSummary = 'WeatherSummary'

lbApparentTemperature = 'ApparentTemperature_F'
lbPressure = 'Pressure_hPa'
lbWindSpeed = 'WindSpeed'
lbCloudCover = 'cloudCover'

lbWindBearing = 'WindBearing'

#average intensity of rain fall: could be mm/minute mm/hour, inch per hour, inch per minute
lbPrecipIntensity = 'PrecipIntensity' 	

lbDewPoint = 'dewPoint_F'

# chance of rain
lbPrecipProbability = 'PrecipProbability'

lbDayOfYear='dayOfYear'
lbHourOfDay='hourOfDay'
lbMinuteOfDay='minuteOfDay'


In order to get rid of the small amount of text values in cloudCover, we replace the content with the median value of cloudCover
Furthermore, some values are floats as text, and others floats as floats. We are going to change the type float too



In [4]:
filtered = df[lbCloudCover]
median =  (df[filtered != 'cloudCover'])[lbCloudCover].median()
print('median: ' + str(median))

median: 0.12


In [5]:
df[lbCloudCover].replace('cloudCover', median, inplace=True)

We can now change the type to float

In [6]:
df[lbCloudCover] = df[lbCloudCover].astype(float)

In [7]:
df[lbCloudCover].unique()

array([0.12, 0.75, 0.  , 1.  , 0.31, 0.44, 0.13, 0.19, 0.25, 0.16, 0.21,
       0.15, 0.14, 0.27, 0.28, 0.17, 0.05, 0.1 , 0.26, 0.29, 0.11, 0.09,
       0.06, 0.02, 0.08, 0.04, 0.35, 0.22, 0.23, 0.54, 0.39, 0.03, 0.07,
       0.76, 0.62, 0.18, 0.79, 0.48, 0.24, 0.57, 0.41, 0.78, 0.2 , 0.77,
       0.46, 0.55, 0.01, 0.51, 0.47, 0.5 , 0.4 , 0.3 , 0.43, 0.33, 0.6 ,
       0.68, 0.66, 0.45, 0.34, 0.52, 0.67, 0.49, 0.37, 0.36, 0.61, 0.38,
       0.42, 0.53, 0.63, 0.32, 0.56, 0.58, 0.72, 0.73, 0.71, 0.64, 0.59])

Now let's convert the two text columns to one hot endoded columns
We can use pandas get_dummies method to create one hot encoded columns with a prefix.
We start with 'WeatherIndicator'

In [8]:
df[lbWeatherIndicator].unique()

array(['clear-night', 'partly-cloudy-night', 'clear-day', 'cloudy',
       'partly-cloudy-day', 'rain', 'snow', 'wind', 'fog'], dtype=object)

In [9]:
df = pd.concat([df,pd.get_dummies(df[lbWeatherIndicator], prefix='weatherIndicator')],axis=1)

In [10]:
# now drop the original column (you don't need it anymore)
df.drop([lbWeatherIndicator],axis=1, inplace=True)

Let's do the same for 'WeatherSummary'

In [11]:
df[lbWeatherSummary].unique()

array(['Clear', 'Mostly Cloudy', 'Overcast', 'Partly Cloudy', 'Drizzle',
       'Light Rain', 'Rain', 'Light Snow', 'Flurries', 'Breezy', 'Snow',
       'Rain and Breezy', 'Foggy', 'Breezy and Mostly Cloudy',
       'Breezy and Partly Cloudy', 'Flurries and Breezy', 'Dry',
       'Heavy Snow'], dtype=object)

In [12]:
df = pd.concat([df,pd.get_dummies(df[lbWeatherSummary], prefix='weatherSummary')],axis=1)

In [13]:
# now drop the original column (you don't need it anymore)
df.drop([lbWeatherSummary],axis=1, inplace=True)

So now all our data types are either type int or type float

In [14]:
df.dtypes

Unnamed: 0                                   int64
Timestamp                                    int64
TotalUsage_kW                              float64
Generated_kW                               float64
HouseOverall_kW                            float64
DishWasher_kW                              float64
Furnace1_kW                                float64
Furnace2_kW                                float64
HomeOffice_kW                              float64
Fridge_kW                                  float64
WineCellar_kW                              float64
GarageDoor_kW                              float64
KitchenDevice12_kW                         float64
KitchenDevice14_kW                         float64
KitchenDevice38_kW                         float64
Barn_kW                                    float64
Well_kW                                    float64
Microwave_kW                               float64
Living room_kW                             float64
Solar_kW                       

As described in part I, we need to convert the timesamp int value to proper 1 minute step values
The first item starts at '2016-01-01 05:00:00'

In [15]:
df[lbTimestamp] = pd.DatetimeIndex(pd.date_range('2016-01-01 05:00', periods=len(df),  freq='min'))
print(df[lbTimestamp].min())
print(df[lbTimestamp].max())

2016-01-01 05:00:00
2016-12-16 03:29:00


We create new features dayOfYear, hourOfDay and minuteOfDay

In [16]:
df[lbDayOfYear] = df[lbTimestamp].apply(lambda x : x.dayofyear)
df[lbHourOfDay] = df[lbTimestamp].apply(lambda x : x.hour)
df[lbMinuteOfDay] = df[lbTimestamp].apply(lambda x : x.minute + 60*x.hour)


In [17]:
print('day of year examples:' + str(df[lbDayOfYear].iloc[0+3])+ ','+ str(df[lbDayOfYear].iloc[20 * 1440+ 8*60 + 5]))
print('hour of day examples:' + str(df[lbHourOfDay].iloc[0+3])+ ','+ str(df[lbHourOfDay].iloc[20 * 1440 + 8*60 + 5]))
print('minute of day examples:' + str(df[lbMinuteOfDay].iloc[0+3])+ ','+ str(df[lbMinuteOfDay].iloc[20 * 1440 + 8*60 + 5]))


day of year examples:1,21
hour of day examples:5,13
minute of day examples:303,785


Removing outliers

In [18]:
outlierFilter = 2
#median = float(df[lbTotalEneryUsage].median())

df[lbTotalEneryUsage] = np.where(df[lbTotalEneryUsage] > outlierFilter, outlierFilter, df[lbTotalEneryUsage])

In [19]:
df[lbTotalEneryUsage].max()

2.0

In [20]:
outlierFilter = 16
df[lbWindSpeed] = np.where(df[lbWindSpeed] > outlierFilter, outlierFilter, df[lbWindSpeed])

In [21]:
df[lbWindSpeed].max()

16.0

In [22]:
outlierFilterMax = 1030
outlierFilterMin = 1000
df[lbPressure] = np.where(df[lbPressure] > outlierFilterMax, outlierFilterMax, df[lbPressure])
df[lbPressure] = np.where(df[lbPressure] < outlierFilterMin, outlierFilterMin, df[lbPressure])

In [23]:
print(str(df[lbPressure].max()))
print(str(df[lbPressure].min()))


1030.0
1000.0


Now lets create normalized columns of our features of interest


In [24]:
from sklearn import preprocessing
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()


In [25]:
x = df[[lbTemperature]].values.astype(float)
# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df[lbTemperature+'_normalized'] = pd.DataFrame(x_scaled)

In [26]:
from sklearn import preprocessing
x = df[[lbWindSpeed]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbWindSpeed+'_normalized'] = pd.DataFrame(x_scaled)

In [27]:
from sklearn import preprocessing
x = df[[lbHumidity]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbHumidity+'_normalized'] = pd.DataFrame(x_scaled)

In [28]:
from sklearn import preprocessing
x = df[[lbPressure]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbPressure+'_normalized'] = pd.DataFrame(x_scaled)

In [29]:
from sklearn import preprocessing
x = df[[lbCloudCover]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbCloudCover+'_normalized'] = pd.DataFrame(x_scaled)

In [30]:
from sklearn import preprocessing
x = df[[lbWindBearing]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbWindBearing+'_normalized'] = pd.DataFrame(x_scaled)

In [31]:
from sklearn import preprocessing
x = df[[lbPrecipIntensity]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbPrecipIntensity+'_normalized'] = pd.DataFrame(x_scaled)

In [32]:
from sklearn import preprocessing
x = df[[lbDewPoint]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbDewPoint+'_normalized'] = pd.DataFrame(x_scaled)

In [33]:
from sklearn import preprocessing
x = df[[lbDayOfYear]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbDayOfYear+'_normalized'] = pd.DataFrame(x_scaled)

In [34]:
from sklearn import preprocessing
x = df[[lbHourOfDay]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbHourOfDay+'_normalized'] = pd.DataFrame(x_scaled)

In [35]:
from sklearn import preprocessing
x = df[[lbMinuteOfDay]].values.astype(float)
x_scaled = min_max_scaler.fit_transform(x)
df[lbMinuteOfDay+'_normalized'] = pd.DataFrame(x_scaled)

In [36]:
print(df.dtypes[0:32])
print(df.dtypes[32:])


Unnamed: 0                             int64
Timestamp                     datetime64[ns]
TotalUsage_kW                        float64
Generated_kW                         float64
HouseOverall_kW                      float64
DishWasher_kW                        float64
Furnace1_kW                          float64
Furnace2_kW                          float64
HomeOffice_kW                        float64
Fridge_kW                            float64
WineCellar_kW                        float64
GarageDoor_kW                        float64
KitchenDevice12_kW                   float64
KitchenDevice14_kW                   float64
KitchenDevice38_kW                   float64
Barn_kW                              float64
Well_kW                              float64
Microwave_kW                         float64
Living room_kW                       float64
Solar_kW                             float64
Temperature_F                        float64
Humidity                             float64
Visibility

Now lets save our post ETL dataset to a new csv file.


In [ ]:
from project_lib import Project
project = Project(None,"[GUID]","p-[GUID]")
project.save_data(file_name = "smart-home-dataset-with-weather-information_post_feature_eng.csv",data = df.to_csv(index=False))

We have now extracted transformed and loaded our data. We can now continue with feature creation in part III